In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from datetime import datetime, date, time,timedelta
import pickle
import sqlite3
from sqlite3 import Error
import datetime

In [ ]:
import socketio
import json
import decision_tree

#ip = 'http://35.246.29.217:65080/' 
#ip = 'http://146.169.173.109:65080/' 
ip = 'http://192.168.0.27:65080'

sio = socketio.Client()

@sio.on('connect')
def on_connect():
    print('connection established')

@sio.on('new message')
def on_message(data):
    data = json.loads(data['message'])
    hour = data['hour']
    minute = data['minute']
    second = data['second']
    # call function to increase level
    print(hour)
    print(minute)
    print(second)
    t = DecisionTree(hour,minute)
    level = t.analysis()
    print('predicted level: ', level)


@sio.on('disconnect')
def on_disconnect():
    print('disconnected from server')

sio.connect(ip)
sio.wait()
#sio.sleep(1)

connection established
09
35
08
predicted level:  [3.]
08
35
08
predicted level:  [2.]


In [40]:
class DecisionTree():
    def __init__(self, hour,minute):
        self.hour = hour
        self.minute = minute
        
    def create_connection(self,db_file):
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None
    
    def get_db_Time(self,conn):
        cur = conn.cursor()
        cur.execute("SELECT Wake, Breakfast, Lunch, Dinner, Sleep From questions WHERE Name = 'Louis Kueh'")
        rows = cur.fetchall()
        return rows

    def get_db_daily_time(self,conn):
        cur = conn.cursor()
        #cur.execute("SELECT Timestamp From timeTaken")
        cur.execute("SELECT Timestamp From timeTaken WHERE BoxNo = 1")
        rows = cur.fetchall()
        return rows
    
    def get_init(self):
        database = './storage.db'
        conn = self.create_connection(database)
        with conn:
            init_time = self.get_db_daily_time(conn)
        
        actual_time = [x[0][3:24] for x in init_time]
        
        t = (actual_time[0].replace('Mar','03').replace(' ','/'))[12:]

        year = '19'
        month = '01'
        day = '01'
        t = year+'/'+month+'/'+day+ ' '+t
        actual_time_convert = datetime.datetime.strptime(t, "%y/%m/%d %H:%M:%S")
        #print('actual_time_convert', actual_time_convert)
        return actual_time_convert
    
    def analysis(self):
        year = '19'
        month = '01'
        day = '01'
        time = year+'/'+month+'/'+day+ ' ' + self.hour + ':' + self.minute
        test_time = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        print('test_time ', test_time)
        init_time = self.get_init() #get initial setting time to take the medcine
        diff = abs(test_time-init_time) # find difference between actual and pre-set
        print(diff)
        x_in = np.array([(diff.total_seconds()/60)]) # convert to minutes
        
        # Load model for testing
        filename = 'notification_model.sav'
        model = pickle.load(open(filename, 'rb'))
        y_predict = model.predict(x_in.reshape(-1,1))
        
        return y_predict # notification level

t = DecisionTree('9','55')
level = t.analysis()
print(level)

test_time  2019-01-01 09:55:00
1:39:44
[3.]


# --------------
### Based on init

In [44]:
class DecisionTree():
    def __init__(self, hour,minute):
        self.hour = hour
        self.minute = minute
        
    def create_connection(self,db_file):
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None
    
    def get_db_Time(self,conn):
        cur = conn.cursor()
        cur.execute("SELECT Wake, Breakfast, Lunch, Dinner, Sleep From questions WHERE Name = 'Louis Kueh'")
        rows = cur.fetchall()
        return rows

    def get_db_daily_time(self,conn):
        cur = conn.cursor()
        cur.execute("SELECT Timestamp From timeTaken")
        rows = cur.fetchall()
        return rows
    
    def get_init(self):
        database = './storage.db'
        conn = create_connection(database)
        with conn:
            init_time = get_db_Time(conn)
        year = '19'
        month = '01'
        day = '01'
        time = year+'/'+month+'/'+day+ ' ' + init_time[0][0]
        init_wake = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        # print(init_wake)

        time = year+'/'+month+'/'+day+ ' ' + init_time[0][1]
        init_morning = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        # print(init_morning)

        time = year+'/'+month+'/'+day+ ' ' + init_time[0][2]
        init_noon = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        # print(init_noon)

        time = year+'/'+month+'/'+day+ ' ' + init_time[0][3]
        init_evening = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        # print(init_evening)

        time = year+'/'+month+'/'+day+ ' ' + init_time[0][4]
        init_sleep = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        # print(init_sleep)

        init = [init_wake,init_morning,init_noon,init_evening,init_sleep]
        print(init[1])
        return init
    
    def analysis(self):
        year = '19'
        month = '01'
        day = '01'
        time = year+'/'+month+'/'+day+ ' ' + self.hour + ':' + self.minute
        test_time = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        
        init_time = self.get_init() #get initial setting time to take the medcine
        diff = test_time-init_time[1] # find difference between actual and pre-set
        x_in = np.array([(diff.total_seconds()/60)]) # convert to minutes
        
        # Load model for testing
        filename = 'notification_model.sav'
        model = pickle.load(open(filename, 'rb'))
        y_predict = model.predict(x_in.reshape(-1,1))
        
        return y_predict # notification level
    
t = DecisionTree('8','55')
level = t.analysis()
print(level)

2019-01-01 09:00:00
[1.]


# -------------

In [7]:
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None

def get_db_Time(conn):
    cur = conn.cursor()
    cur.execute("SELECT Wake, Breakfast, Lunch, Dinner, Sleep From questions WHERE Name = 'Louis Kueh'")
 
    rows = cur.fetchall()
    return rows

def get_db_daily_time(conn):
    cur = conn.cursor()
    cur.execute("SELECT Timestamp From timeTaken")
 
    rows = cur.fetchall()
    return rows

# resources/storage.db
database = './storage.db'
conn = create_connection(database)
with conn:
    init_time = get_db_Time(conn)
    daily_time = get_db_daily_time(conn)


print(init_time)
print(daily_time)

[('07:34', '8:00', '12:00', '18:34', '22:45')]
[('Mon Mar 11 2019 10:50:16 GMT+0000 (GMT Standard Time)',), ('Mon Mar 11 2019 10:50:56 GMT+0000 (GMT Standard Time)',)]


In [13]:
database = './storage.db'
conn = create_connection(database)
with conn:
    init_time = get_db_daily_time(conn)

print(init_time)
actual_time = [x[0][3:24] for x in get_db_daily_time]
print(actual_time)

t = (actual_time[0].replace('Mar','03').replace(' ','/'))
print(t)
t = (t[1:7] + t[9:11] + ' ' + t[12:])
print(t)

print((actual_time[0]))
year = '19'
month = '05'
day = '05'

actual_time_convert = datetime.datetime.strptime(t, "%m/%d/%y %H:%M:%S")
print(actual_time_convert)

[('Mon Mar 11 2019 10:50:16 GMT+0000 (GMT Standard Time)',), ('Mon Mar 11 2019 10:50:56 GMT+0000 (GMT Standard Time)',)]


TypeError: 'function' object is not iterable

In [11]:
year = '19'
month = '01'
day = '01'
time = year+'/'+month+'/'+day+ ' ' + init_time[0][0]
init_wake = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
# print(init_wake)

time = year+'/'+month+'/'+day+ ' ' + init_time[0][1]
init_morning = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
# print(init_morning)

time = year+'/'+month+'/'+day+ ' ' + init_time[0][2]
init_noon = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
# print(init_noon)

time = year+'/'+month+'/'+day+ ' ' + init_time[0][3]
init_evening = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
# print(init_evening)

time = year+'/'+month+'/'+day+ ' ' + init_time[0][4]
init_sleep = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
# print(init_sleep)

init = [init_wake,init_morning,init_noon,init_evening,init_sleep]

print(init[0].time())
print(init[1].time())
print(init[2].time())
print(init[3].time())
print(init[4].time())

07:34:00
08:00:00
12:00:00
18:34:00
22:45:00


In [ ]:
#'''Daily questionnaires'''
questionnaires = './que.csv'
raw = read_data(questionnaires)
raw_data = raw[1:,1:]
title = raw[0,1:]
X_que = raw_data.copy()
X_que = X_que.astype(float)

#'''Initial setting'''
init = './init.csv' # morning, afternoon, evening, wake, sleep
raw = read_data(init)
raw_data = raw[1:,:]

X_init = raw_data.copy()
#X_init = X_init.astype(float)

#'''Reminder Ranking wrt daily questionnaires'''
notifi = './notification_lvl.csv'
raw = read_data(notifi)
raw_data = raw[1:,1:]

y_notifi = raw_data.copy()
y_notifi = y_notifi.astype(float)

In [ ]:
#X_init[0,0]
import datetime
year = '19'
month = '01'
day = '01'
time = year+'/'+month+'/'+day+ ' ' + X_init[0,0]
init_morning = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
print(init_morning)
time = year+'/'+month+'/'+day+ ' ' + X_init[0,1]
init_afternoon = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,2]
init_evening = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,3]
init_wake = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,4]
init_sleep = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

init = [init_morning,init_afternoon,init_evening,init_wake,init_sleep]

notification_time = []
for i,t in enumerate(init):
    notification_time.append((t+timedelta(minutes = mean_X[i])).time() )# days, seconds, then other fields.
    
res = []
for i in range(len(notification_time)):
    res.append(notification_time[i])

print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])